In [ ]:
import gempy as gp
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from geometron.geometries import gdf_to_points_gdf
# Setting options
np.random.seed(1515)
pd.set_option('precision', 2)

Importation of th geopackage into a geodataframe

In [ ]:
gdf = gpd.read_file('../../../../data/Parties exploitées - Caillette.gpkg', layer = 'Parties exploitées - Caillette')

Rename of the column "Couche" into "layer"

In [ ]:
gdf.rename({'Couche ': 'layer'}, axis=1, inplace=True)

Research and selection of the 2 main layers because others are less important and too isolated

In [ ]:
gdf = gdf.query('layer=="10 Paumes" or layer=="Caillette"')

Creation of a rectangle which includes the points of the 2 layers

In [ ]:
from shapely.geometry import Polygon
aoi = Polygon([[153400, 122500], [156400, 122500], [156400, 124250], [153400, 124250], [153400, 122500]])
aoi

Clip of the rectangle on the geodataframe

In [ ]:
gdf = gpd.clip(gdf, aoi, True) 

Plot of the geodataframe

In [ ]:
gdf.plot(column='layer', legend=True)

Creation of a geodatframe simplier than the first one with only the half of the points because it's enough

In [ ]:
gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.simplify(10))

Plot of the second geodataframe

In [ ]:
gdf.plot(column='layer')

Creation of the points from the geodataframe

In [ ]:
gdf_points = gdf_to_points_gdf(gdf, columns=['layer'])
gdf_points.tail()

Conversion of the geodateframe into a csv file

In [ ]:
gdf.to_csv('Parties exploitées - Caillette.csv')

Creation of the model with boundaries

In [ ]:
geo_model = gp.create_model('Parties exploitées - Caillette')
geo_model = gp.init_data(geo_model, extent=[gdf_points.bounds.minx.min(), gdf_points.bounds.maxx.max(), gdf_points.bounds.miny.min(),  gdf_points.bounds.maxy.max(), gdf_points.geometry.z.min(), gdf_points.geometry.z.max()], resolution=[100, 100, 40])

In [ ]:
gp.set_interpolator(geo_model, theano_optimizer='fast_compile', verbose=[])

Creation of the surfaces in the right order (from the younger to the older)

In [ ]:
geo_model.add_surfaces(['Caillette', '10 Paumes','basement'])

Creation of the surfaces points passing through each point for each layer

In [ ]:
for idx, row in gdf_points.iterrows():
    geo_model.add_surface_points(row.geometry.x, row.geometry.y,row.geometry.z, surface=row.layer)

In [ ]:
geo_model.surfaces

Creation of the orientation of each points and layers

Here we don't have enough time to find how calculate the right orientation from points and surface point so we put a vertical orientation

In [ ]:
for idx, row in gdf_points.iterrows():
    geo_model.add_orientations(row.geometry.x, row.geometry.y,row.geometry.z, surface=row.layer, pole_vector=(0, 0, 1))

In [ ]:
geo_model.orientations

Compilation of the model

In [ ]:
gp.compute_model(geo_model)

Plotting of the model

Exportation of the model in vtk format

In [ ]:
gp._plot.export_to_vtk(geo_model, './tmp_files/Caillette-10_Paumes')